In [6]:
pip install unidecode
pip install selenium
pip install bs4

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [28]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import os
from time import sleep
from selenium.webdriver.common.by import By

def initDriver():

    WINDOW_SIZE = "1000,2000"
    chrome_options = Options()
    #chrome_options.add_argument("--headless")
    chrome_options.add_argument("--window-size=%s" % WINDOW_SIZE)
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('disable-infobars')
    chrome_options.add_argument('--disable-gpu') if os.name == 'nt' else None  # Windows workaround
    chrome_options.add_argument("--verbose")
    chrome_options.add_argument("--no-default-browser-check")
    chrome_options.add_argument("--ignore-ssl-errors")
    chrome_options.add_argument("--allow-running-insecure-content")
    chrome_options.add_argument("--disable-web-security")
    chrome_options.add_argument("--disable-feature=IsolateOrigins,site-per-process")
    chrome_options.add_argument("--no-first-run")
    chrome_options.add_argument("--disable-notifications")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disable-translate")
    chrome_options.add_argument("--ignore-certificate-error-spki-list")
    chrome_options.add_argument("--ignore-certificate-errors")
    chrome_options.add_argument("--disable-blink-features=AutomationControllered")
    chrome_options.add_experimental_option('useAutomationExtension', False)
    prefs = {"profile.default_content_setting_values.notifications": 2}
    chrome_options.add_experimental_option("prefs", prefs)
    chrome_options.add_argument("--start-maximized")  # open Browser in maximized mode
    chrome_options.add_argument("--disable-dev-shm-usage")  # overcome limited resource problems
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option("prefs", {"profile.managed_default_content_settings.images": 2})
    chrome_options.add_argument('disable-infobars')

    driver = webdriver.Chrome(options=chrome_options)
    return driver


In [29]:

driver = initDriver()


In [30]:
from selenium.webdriver.common.keys import Keys
keyword = "vụ tấn công ở đắk lắk"
keyword = "tổng bí thư trung quốc đến thăm việt nam"
driver.get("https://www.google.com/search?sca_esv=599503376&sxsrf=ACQVn0-YkRlT1UNLhRnMzK7gcVba-gwsLQ:1705596004849&tbm=nws&q="+keyword)
# inputbox = driver.find_element(By.XPATH,"//textarea[@id='APjFqb']")
# inputbox.send_keys()
# inputbox.send_keys(Keys.ENTER)
# for i in range(10):
#     driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")



In [31]:

# def get_info(item):
#     link = item.find_element(By.XPATH,"//a[@jsname='UWckNb']").get_attribute("href")
#     name = item.find_element(By.XPATH,"//h3[@class='LC20lb MBeuO DKV0Md']").text
#     time = item.find_element(By.XPATH,"//span[@class='LEwnzc Sqrs4e']").text
#     des= item.find_element(By.XPATH,"//div[@class='VwiC3b yXK7lf lVm3ye r025kc hJNv6b Hdw6tb']/span[2]").text



In [32]:
class GoogleResult:
    def __init__(self):
        self.name = None
        self.link = None
        self.description = None
    def show(self):
        print("link : ",self.link)
        print("name : ",self.name)
        print("description : ",self.description)
    def returnjson(self):
        return {"name":self.name,
                "link":self.link,
                "description":self.description
                }
        
def get_info(item):
    obj = GoogleResult()
    
# def get_info(item):
#     link = item.find_element(By.XPATH,"//a[@jsname='UWckNb']").get_attribute("href")
#     name = item.find_element(By.XPATH,"//h3[@class='LC20lb MBeuO DKV0Md']").text
#     time = item.find_element(By.XPATH,"//span[@class='LEwnzc Sqrs4e']").text
#     des= item.find_element(By.XPATH,"//div[@class='VwiC3b yXK7lf lVm3ye r025kc hJNv6b Hdw6tb']/span[2]").text

    try:
        obj = GoogleResult()
        obj.link =  item.find('a')['href']
        obj.name = item.find('div',{'class','n0jPhd ynAwRc MBeuO nDgy9d'}).text
        obj.time = item.find('div',{'class','OSrXXb rbYSKb LfVVr'}).text
        obj.description = item.find('div',{'class','GI74Re nDgy9d'}).text
        
    except Exception as e :
        print(e)
    
    print("\n---------\n")
    obj.show()
    return obj


In [33]:
from bs4 import BeautifulSoup
html_source = driver.page_source

soup = BeautifulSoup(html_source, 'html.parser')

In [34]:
from bs4 import BeautifulSoup
import time 

data = []
for _ in range(10):
    html_source = driver.page_source

    soup = BeautifulSoup(html_source, 'html.parser')

    listall = soup.findAll('div', {'class','SoaBEf'})    

    for item in listall:
        temp = get_info(item)
        data.append(temp.returnjson())
        
    url = soup.find_all('td',{'class','d6cvqb BBwThe'})[1].find('a')['href']
    driver.get("https://www.google.com/"+url)
    
        
    


---------

link :  https://vov.vn/chinh-tri/dien-mung-ky-niem-74-nam-thiet-lap-quan-he-ngoai-giao-viet-nam-trung-quoc-post1072516.vov
name :  Điện mừng kỷ niệm 74 năm thiết lập quan hệ ngoại giao Việt Nam-Trung Quốc
description :  VOV.VN - Tổng Bí thư Nguyễn Phú Trọng và Chủ tịch nước Võ Văn Thưởng đã 
trao đổi điện mừng với Tổng Bí thư, Chủ tịch nước Trung Quốc Tập Cận...

---------

link :  https://dantri.com.vn/xa-hoi/tong-bi-thu-nguyen-phu-trong-va-tong-bi-thu-tap-can-binh-trao-doi-dien-mung-20240118204025686.htm
name :  Tổng Bí thư Nguyễn Phú Trọng và Tổng Bí thư Tập Cận Bình trao đổi điện mừng
description :  (Dân trí) - Lãnh đạo Đảng, Nhà nước, Chính phủ và Quốc hội hai nước Việt 
Nam và Trung Quốc trao đổi điện mừng, nhân kỷ niệm 74 năm ngày...

---------

link :  https://laodong.vn/thoi-su/lanh-dao-viet-nam-trung-quoc-trao-doi-dien-mung-74-nam-thiet-lap-quan-he-1294220.ldo
name :  Lãnh đạo Việt Nam-Trung Quốc trao đổi điện mừng 74 năm thiết lập quan hệ
description :  Lãnh đạo 

In [35]:
data

[{'name': 'Điện mừng kỷ niệm 74 năm thiết lập quan hệ ngoại giao Việt Nam-Trung Quốc',
  'link': 'https://vov.vn/chinh-tri/dien-mung-ky-niem-74-nam-thiet-lap-quan-he-ngoai-giao-viet-nam-trung-quoc-post1072516.vov',
  'description': 'VOV.VN - Tổng Bí thư Nguyễn Phú Trọng và Chủ tịch nước Võ Văn Thưởng đã \ntrao đổi điện mừng với Tổng Bí thư, Chủ tịch nước Trung Quốc Tập Cận...'},
 {'name': 'Tổng Bí thư Nguyễn Phú Trọng và Tổng Bí thư Tập Cận Bình trao đổi điện mừng',
  'link': 'https://dantri.com.vn/xa-hoi/tong-bi-thu-nguyen-phu-trong-va-tong-bi-thu-tap-can-binh-trao-doi-dien-mung-20240118204025686.htm',
  'description': '(Dân trí) - Lãnh đạo Đảng, Nhà nước, Chính phủ và Quốc hội hai nước Việt \nNam và Trung Quốc trao đổi điện mừng, nhân kỷ niệm 74 năm ngày...'},
 {'name': 'Lãnh đạo Việt Nam-Trung Quốc trao đổi điện mừng 74 năm thiết lập quan hệ',
  'link': 'https://laodong.vn/thoi-su/lanh-dao-viet-nam-trung-quoc-trao-doi-dien-mung-74-nam-thiet-lap-quan-he-1294220.ldo',
  'description':

In [36]:
driver.quit()

In [5]:
from unidecode import unidecode
keyword = "tổng bí thư trung quốc đến thăm việt nam"
fileoutput = unidecode(keyword).replace(" ","_")

print(fileoutput)

tong_bi_thu_trung_quoc_den_tham_viet_nam
